In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

# Python Chess Erweiterung

Dieses Notebook erweitert die `chess`-Bibliothek um eine inkrementelle Transpositionstabelle. In dieser Tabelle wird für jede Position die Anzahl der bisher bekannten Transpositionen gespeichert. Eine Transposition liegt genau dann vor, wenn eine gegebene Stellung auch über eine andere Zugfolge erreichbar ist.  
Beispiel: Wenn die aktuelle Stellung `x` im Spielverlauf bereits drei Mal auftrat, ist dies die zweite Wiederholung, d. h. die zweite Transposition. Die Transpositionstabelle enthält für dieses Board dann den Wert `2`.

Mithilfe der inkrementellen Transpositionstabelle ist die Prüfung auf Wiederholungen (z. B. bei der `fivefold-repetition`-Regel) deutlich effizienter möglich und es muss nicht jedes Mal der gesamte Spielverlauf untersucht werden. Die originale Implementierung in der `chess`-Bibliothek (ohne inkrementelle Transpositionstabelle) ist [hier](https://python-chess.readthedocs.io/en/latest/_modules/chess.html#Board.is_repetition) zu finden.

Die folgenden Methoden der `chess`-Bibliothek werden überschrieben:
- `__init__`: Die Transpositionstabelle wird mit der Start-Position initialisiert.
- `set_fen`: Die Transpositionstabelle wird mit der gewählten Position neu initialisiert.
- `push`: Aktualisierung der Transpositionstabelle nach einem neuen Zug.
- `pop`: Aktualisierung der Transpositionstabelle nach dem Entfernen des letzten Zuges.
- `is_repetition`: Effizientere Implementierung der Elternmethode mithilfe der inkrementellen Transpositionstabelle. In der Elternklasse wird hier der gesamte Stack durchsucht.

In [ ]:
from chess import *
import chess.pgn as pgn

class Board(Board):  # type: ignore

    def __init__(self, fen: Optional[str] = STARTING_FEN, *, chess960: bool = False) -> None:
        super().__init__(fen, chess960=chess960)
        self._transposition_table: Dict[Hashable, int] = {self._transposition_key(): 0}

    def set_fen(self, fen: str) -> None:
        super().set_fen(fen)
        self._transposition_table = {self._transposition_key(): 0}

    def push(self, move: Move) -> None:
        super().push(move)
        key = self._transposition_key()
        occurences = self._transposition_table.get(key, -1)
        self._transposition_table[key] = occurences + 1

    def pop(self) -> Move:
        key = self._transposition_key()
        occurences = self._transposition_table[key]
        if occurences == 0:
            del self._transposition_table[key]
        else:
            self._transposition_table[key] = occurences - 1
        return super().pop()

    def is_repetition(self, count: int = 3) -> bool:
        return self._transposition_table[self._transposition_key()] >= count - 1

# Debugging Bereich

In [ ]:
board = Board()
board._transposition_table